# Information Retrieval and Text Analytics Project

## Introduction

**Overview of Information Retrieval (IR) Systems:** An information retrieval system is a system or software designed to search for and retrieve relevant information from a large collection of unstructured or semi-structured data, such as text documents, images, or multimedia content, based on user queries. These systems are essential in handling vast amounts of information, such as those found on the internet, in digital libraries, or within enterprise data systems.

Examples of information retrieval systems:
1.   **Google:** A search engine that retrieves relevant web pages based on keywords entered by the user.
2.   **Amazon:** An e-commerce platform where users search for products by entering keywords or browsing categories.
3.   **PubMed:** A database for medical research articles, allowing users to search for academic papers related to health and science.



**Background:** Data can be found everywhere around us. With the proliferation of digital devices, vast amounts of data are being generated and shared. We can find valuable information in news, books, papers, documentations and wikis. Though to find this valuable information, one must swim through a sea of redundant and sometimes useless data. Therefore, there is a growing need for a way to quickly obtain and sift through this data to extract valuable insights. Many information retrieval models were developed to handle said issue, albeit with varying performances. This creates the issue where we want to know which model performs better in the realm of information retrieval.



**Objective:** This project aims to develop a system that retrieves relevant information from text datasets using 3 different information retrieval models and comparing between their performances to know which one is better. The system also allows us to improve the understanding and organization of the data through text analytics. This includes leveraging preprocessing techniques to enhance text representation, applying robust retrieval methods to ensure accuracy, and utilizing visualizations to provide actionable insights and evaluate performance effectively.

**Scope:** The scope includes implementing advanced preprocessing techniques such as tokenization, case standardization, stopwords removal, stemming, and TF-IDF to improve text representation. Additionally, it encompasses the integration of Vector Space Model, Boolean Retrieval Model, and BM25 retrieval algorithms for ranking and identifying relevant information based on user queries. We compare between the performances of these algorithms. To enhance usability, the project will also incorporate visualization tools such as word clouds for top keywords in documents, Frequency distribution of words, document-query similarity scores (e.g., bar charts), and clustering topics using LDA (Latent Dirichlet Allocation). The project will be limited to textual data and will not cover multimedia or non-textual information retrieval.

## Data Collection

In [ ]:
# Downloading required dataset(s)
!pip install kagglehub
!pip install rank_bm25
!pip install nltk
!pip install scikit-learn
!pip install wordcloud
!pip install pyLDAvis
!pip install pyLDAvis.sklearn
import kagglehub
import nltk
import os
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from rank_bm25 import BM25Okapi
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
nltk.download('punkt_tab')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
# path = kagglehub.dataset_download("crawford/20-newsgroups")
# The dataset will be download in /home/<user>/.cache/kagglehub


## Data Inspection

In [ ]:
# Specify the file path
file_path = "./20-newsgroups/talk.politics.guns.txt"  # Replace with the actual path to your file

# Check if the file exists
if os.path.exists(file_path):
    # Open the file and read its contents
    with open(file_path, 'r', encoding='latin-1') as file:  # Use 'latin-1' encoding
        file_contents = file.read()

    # Print or process the file contents
    print(file_contents[:200])  # Print the first 200 characters of the file contents
else:
    print(f"Error: File not found at '{file_path}'")

# Data Preprocessing

#### Implementing tokenization, lowercasing, stopwords removal, stemming and lemmatization.

In [6]:
def preprocess_text(text):
    # Tokenization
    tokens = nltk.word_tokenize(text)

    # Lowercasing
    tokens = [token.lower() for token in tokens]

    # Stopword Removal
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words and token.isalnum()]

    # Stemming
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(token) for token in tokens]

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]

    return ' '.join(tokens)

#### Implementing Vectorization (BoW/TF-IDF)

In [7]:
def vectorize_texts(texts, method='bow'):
    """
    Vectorize texts using Bag of Words (BOW) or TF-IDF.

    Parameters:
    texts (list of str): List of texts to vectorize.
    method (str): Method of vectorization ('bow' or 'tfidf').

    Returns:
    X (sparse matrix): Vectorized text data.
    vectorizer (Vectorizer object): Fitted vectorizer.
    """
    if method == 'bow':
        vectorizer = CountVectorizer()
    elif method == 'tfidf':
        vectorizer = TfidfVectorizer()
    else:
        raise ValueError("Method must be 'bow' or 'tfidf'")

    X = vectorizer.fit_transform(texts)
    return X, vectorizer

##### Read and Process all the documents

In [8]:
def read_and_preprocess_files(directory_path='./20-newsgroups'):
    preprocessed_texts = []
    filenames = []
    for filename in os.listdir(directory_path):
        file_path = os.path.join(directory_path, filename)
        if os.path.isfile(file_path):
            with open(file_path, 'r', encoding='latin-1') as file:
                text = file.read()
                preprocessed_text = preprocess_text(text)
                preprocessed_texts.append(preprocessed_text)
                filenames.append(filename)
    return preprocessed_texts, filenames

In [ ]:
preprocessed_texts, filenames = read_and_preprocess_files()
preprocessed_texts[:][0][:50]

#### BoW Vectorization

In [ ]:
X_bow, vectorizer_bow = vectorize_texts(preprocessed_texts, method='bow')
print(X_bow)

#### TF-IDF Vectorization

In [ ]:
X_tfidf, vectorizer_tfidf = vectorize_texts(preprocessed_texts, method='tfidf')
print(X_tfidf)

# Retrieval Models

##### Set the query

In [12]:
query = "god will forgive you"

## Implementing BM25 (Best Matching 25)

In [13]:
def apply_bm25(query=query, preprocessed_texts=preprocessed_texts, filenames=filenames):
    """
    Apply BM25 algorithm to retrieve and rank documents based on a query.
    Parameters:
    query (str): The search query.
    preprocessed_texts (list of str): List of preprocessed texts.
    filenames (list of str): List of filenames corresponding to the texts.
    Returns:
    list: List of retrieved document filenames.
    dict: Dictionary of filenames and their BM25 scores.
    """
    # Tokenize the preprocessed texts
    tokenized_texts = [text.split() for text in preprocessed_texts]

    # Initialize BM25 with the tokenized corpus
    bm25 = BM25Okapi(tokenized_texts)

    # Preprocess and tokenize the query
    tokenized_query = preprocess_text(query).split()

    # Get BM25 scores for the query
    scores = bm25.get_scores(tokenized_query)

    # Rank documents based on the scores
    top_n = 5
    top_n_indices = scores.argsort()[-top_n:][::-1]

    # Retrieve the filenames of the top-ranked documents
    retrieved_docs = [filenames[i] for i in top_n_indices]

    # Create a dictionary of filenames and their scores
    ranked_documents = {filenames[i]: scores[i] for i in top_n_indices}

    return retrieved_docs, ranked_documents

In [14]:
bm25_retrieved_docs, bm25_ranked_documents = apply_bm25(query=query)

In [ ]:
bm25_ranked_documents

## Implementing Vector Space Model (VSM)

In [16]:
def apply_vsm(query=query, preprocessed_texts=preprocessed_texts, vectorizer=vectorizer_bow):
    # Vectorize the preprocessed texts
    X = vectorizer.fit_transform(preprocessed_texts)

    # Vectorize the query
    query_vector = vectorizer.transform([query])

    # Calculate cosine similarity between the query and the documents
    similarities = cosine_similarity(query_vector, X).flatten()

    # Rank documents based on similarity scores
    ranked_indices = similarities.argsort()[::-1]
    ranked_indices = ranked_indices[:5]

    # Print the ranked documents with filenames and similarity scores
    dict = {}
    for i in ranked_indices[:]:  # Display top 20 results
        dict[filenames[i]] = similarities[i]
        # print(f"Document: {filenames[i]}, Similarity: {similarities[i]}")

    retrieved_documents = [filenames[i] for i in ranked_indices]
    return retrieved_documents, dict

In [17]:
vsm_bow_retrieved_documents, vsm_bow_ranked_documents = apply_vsm(vectorizer=vectorizer_bow)

In [ ]:
vsm_bow_ranked_documents 

In [19]:
vsm_tfidf_retrieved_documents, vsm_tfidf_ranked_documents = apply_vsm(vectorizer=vectorizer_tfidf)

In [ ]:
vsm_tfidf_ranked_documents

# Evaluation

#### Decide the relevant documents

In [ ]:
def get_documents_names(directory_path='./20-newsgroups'):
    # Create a dictionary of filenames with keys from 1 to 20
    filenames_dict = {i+1: filename for i, filename in enumerate(os.listdir(directory_path)[:20])}

    # Print the dictionary for user reference
    for key, value in filenames_dict.items():
        print(f"{key}: {value}")

    return filenames_dict

filenames_dict = get_documents_names()

In [ ]:
def get_relevant_documents(filenames_dict):
    # Get user input for relevant document numbers
    relevant_numbers = input("Enter the numbers related to the relevant documents, separated by commas: ")
    relevant_numbers = [int(num.strip()) for num in relevant_numbers.split(',')]

    # Save the corresponding filenames in a list
    relevant_docs = [filenames_dict[num] for num in relevant_numbers if num in filenames_dict]

    return relevant_docs

# Example usage
relevant_docs = get_relevant_documents(filenames_dict)
print(f"Relevant documents: {relevant_docs}")

### Precesion

In [23]:
def calculate_precision(retrieved_docs, relevant_docs):
    """
    Calculate precision.
    {key: value, key: value}
    Parameters:
    retrieved_docs (list): List of retrieved document filenames.
    relevant_docs (list): List of relevant document filenames.

    Returns:
    float: Precision value.
    """
    # Convert lists to sets for easier calculation
    retrieved_set = set(retrieved_docs)
    relevant_set = set(relevant_docs)

    # Calculate the number of relevant documents retrieved
    relevant_retrieved = retrieved_set.intersection(relevant_set)

    # Calculate precision
    precision = len(relevant_retrieved) / len(retrieved_set) if retrieved_set else 0

    return precision

In [ ]:
# Calculate precision for BM25
bm25_precision = calculate_precision(bm25_retrieved_docs, relevant_docs)
# Calculate precision for VSM with BoW
vsm_bow_precision = calculate_precision(vsm_bow_retrieved_documents, relevant_docs)
# Calculate precision for VSM with TF-IDF
vsm_tfidf_precision = calculate_precision(vsm_tfidf_retrieved_documents, relevant_docs)

print(f"BM25 Precision: {bm25_precision}")
print(f"VSM with BoW Precision: {vsm_bow_precision}")
print(f"VSM with TF-IDF Precision: {vsm_tfidf_precision}")

### Recall

In [25]:
def calculate_recall(retrieved_docs, relevant_docs):
    """
    Calculate recall.

    Parameters:
    retrieved_docs (list): List of retrieved document filenames.
    relevant_docs (list): List of relevant document filenames.

    Returns:
    float: Recall value.
    """
    # Convert lists to sets for easier calculation
    retrieved_set = set(retrieved_docs)
    relevant_set = set(relevant_docs)

    # Calculate the number of relevant documents retrieved
    relevant_retrieved = retrieved_set.intersection(relevant_set)

    # Calculate recall
    recall = len(relevant_retrieved) / len(relevant_set) if relevant_set else 0

    return recall

In [ ]:
# Calculate recall for BM25
bm25_recall = calculate_recall(bm25_retrieved_docs, relevant_docs)
# Calculate recall for VSM with BoW
vsm_bow_recall = calculate_recall(vsm_bow_retrieved_documents, relevant_docs)
# Calculate recall for VSM with TF-IDF
vsm_tfidf_recall = calculate_recall(vsm_tfidf_retrieved_documents, relevant_docs)

print(f"BM25 Recall: {bm25_recall}")
print(f"VSM with BoW Recall: {vsm_bow_recall}")
print(f"VSM with TF-IDF Recall: {vsm_tfidf_recall}")

### Mean Average Precision (MAP)

In [35]:
def calculate_mean_average_precision(retrieved_docs_list, relevant_docs_list):
    """
    Calculate Mean Average Precision (MAP).

    Parameters:
    retrieved_docs_list (list of lists): List of retrieved document filenames for each query.
    relevant_docs_list (list of lists): List of relevant document filenames for each query.

    Returns:
    float: Mean Average Precision value.
    """
    def calculate_average_precision(retrieved_docs, relevant_docs):
        """
        Calculate the average precision for a single query.


        Parameters:
        retrieved_docs (list): List of retrieved document filenames.
        relevant_docs (list): List of relevant document filenames.

        Returns:
        float: Average precision value.
        """
        retrieved_set = set(retrieved_docs)
        relevant_set = set(relevant_docs)
        
        if not relevant_set:
            return 0.0

        relevant_retrieved = 0
        precision_sum = 0.0

        for i, doc in enumerate(retrieved_set):
            if doc in relevant_set:
                relevant_retrieved += 1
                precision_sum += relevant_retrieved / (i + 1)

        return precision_sum / len(relevant_set)

    average_precisions = [
        calculate_average_precision(retrieved_docs, relevant_docs)
        for retrieved_docs, relevant_docs in zip(retrieved_docs_list, relevant_docs_list)
    ]

    return sum(average_precisions) / len(average_precisions) if average_precisions else 0.0

In [ ]:
# Calculate Mean Average Precision
retrieved_docs_list = [bm25_retrieved_docs, vsm_bow_retrieved_documents, vsm_tfidf_retrieved_documents]
relevant_docs_list = [relevant_docs] * 3
mean_average_precision = calculate_mean_average_precision(retrieved_docs_list, relevant_docs_list)
print(f"Mean Average Precision: {mean_average_precision}")

# Visualization

In [ ]:
def count_word_in_files(directory_path='./20-newsgroups', word="god will be with you"):
    """
    Count the occurrences of a word in all files within a directory.

    Parameters:
    directory_path (str): Path to the directory containing text files.
    word (str): Word to count occurrences of.

    Returns:
    dict: A dictionary with filenames as keys and word counts as values, sorted by word counts.
    """
    word_count = {}
    word = word.lower()  # Convert the word to lowercase for case-insensitive matching

    for filename in os.listdir(directory_path):
        file_path = os.path.join(directory_path, filename)
        if os.path.isfile(file_path):
            with open(file_path, 'r', encoding='latin-1') as file:
                text = file.read()

                # Preprocess the text: remove punctuation, convert to lowercase
                text = re.sub(r'[^\w\s]', '', text)
                text = text.lower()

                # Count occurrences of the word
                count = text.split().count(word)
                word_count[filename] = count

    # Sort the dictionary by word counts in descending order
    sorted_word_count = dict(sorted(word_count.items(), key=lambda item: item[1], reverse=True))

    return sorted_word_count

word_counts = count_word_in_files()
print(word_counts)

In [ ]:
from wordcloud import WordCloud

import matplotlib.pyplot as plt

def generate_word_clouds(preprocessed_texts, filenames):
    for text, filename in zip(preprocessed_texts, filenames):
        wordcloud = WordCloud(width=800, height=400, background_color='white').generate(text)
        plt.figure(figsize=(10, 5))
        plt.imshow(wordcloud, interpolation='bilinear')
        plt.axis('off')
        plt.title(f'Word Cloud for {filename}')
        plt.show()

# Generate word clouds for the preprocessed texts
generate_word_clouds(preprocessed_texts, filenames)

In [ ]:
from nltk import FreqDist

import matplotlib.pyplot as plt

# Combine all preprocessed texts into a single string
all_text = ' '.join(preprocessed_texts)

# Tokenize the combined text
tokens = all_text.split()

# Create a frequency distribution
freq_dist = FreqDist(tokens)

# Plot the frequency distribution
plt.figure(figsize=(12, 6))
freq_dist.plot(20, cumulative=False)
plt.title('Frequency Distribution of Top 20 Words')
plt.xlabel('Words')
plt.ylabel('Frequency')
plt.show()

In [ ]:
import matplotlib.pyplot as plt

def plot_similarity_scores(similarity_scores, title):
    """
    Plot similarity scores as a bar chart.

    Parameters:
    similarity_scores (dict): Dictionary of document filenames and their similarity scores.
    title (str): Title of the plot.
    """
    # Extract filenames and scores
    filenames = list(similarity_scores.keys())
    scores = list(similarity_scores.values())

    # Create a bar chart
    plt.figure(figsize=(12, 6))
    plt.barh(filenames, scores, color='skyblue')
    plt.xlabel('Similarity Score')
    plt.ylabel('Document')
    plt.title(title)
    plt.gca().invert_yaxis()  # Invert y-axis to have the highest score on top
    plt.show()

# Plot similarity scores for BM25
plot_similarity_scores(bm25_ranked_documents, 'BM25 Document-Query Similarity Scores')

# Plot similarity scores for VSM with BoW
plot_similarity_scores(vsm_bow_ranked_documents, 'VSM (BoW) Document-Query Similarity Scores')

# Plot similarity scores for VSM with TF-IDF
plot_similarity_scores(vsm_tfidf_ranked_documents, 'VSM (TF-IDF) Document-Query Similarity Scores')

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation

def perform_lda(preprocessed_texts, n_topics=5, n_top_words=10):
    """
    Perform Latent Dirichlet Allocation (LDA) on preprocessed texts.

    Parameters:
    preprocessed_texts (list of str): List of preprocessed texts.
    n_topics (int): Number of topics to extract.
    n_top_words (int): Number of top words to display for each topic.

    Returns:
    None
    """
    # Vectorize the preprocessed texts using TF-IDF
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(preprocessed_texts)

    # Perform LDA
    lda = LatentDirichletAllocation(n_components=n_topics, random_state=42)
    lda.fit(tfidf_matrix)

    # Display the top words for each topic
    feature_names = tfidf_vectorizer.get_feature_names_out()
    for topic_idx, topic in enumerate(lda.components_):
        print(f"Topic #{topic_idx + 1}:")
        print(" ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]))
        print()

# Perform LDA on the preprocessed texts
perform_lda(preprocessed_texts, n_topics=5, n_top_words=10)

# Results and Analysis 

In [ ]:
def present_retrieved_documents(queries, preprocessed_texts, filenames, vectorizer_bow, vectorizer_tfidf):
    for query in queries:
        print(f"Query: {query}")
        
        # Apply BM25
        bm25_retrieved_docs, bm25_ranked_documents = apply_bm25(query=query, preprocessed_texts=preprocessed_texts, filenames=filenames)
        print("BM25 Retrieved Documents:")
        for doc in bm25_retrieved_docs:
            print(f"- {doc}")
        
        # Apply VSM with BoW
        vsm_bow_retrieved_documents, vsm_bow_ranked_documents = apply_vsm(query=query, preprocessed_texts=preprocessed_texts, vectorizer=vectorizer_bow)
        print("VSM (BoW) Retrieved Documents:")
        for doc in vsm_bow_retrieved_documents:
            print(f"- {doc}")
        
        # Apply VSM with TF-IDF
        vsm_tfidf_retrieved_documents, vsm_tfidf_ranked_documents = apply_vsm(query=query, preprocessed_texts=preprocessed_texts, vectorizer=vectorizer_tfidf)
        print("VSM (TF-IDF) Retrieved Documents:")
        for doc in vsm_tfidf_retrieved_documents:
            print(f"- {doc}")
        
        print("\n")

# Sample queries
sample_queries = ["god will punish you", "evolution of species", "quantum mechanics"]

# Present retrieved documents for sample queries
present_retrieved_documents(sample_queries, preprocessed_texts, filenames, vectorizer_bow, vectorizer_tfidf)

In [ ]:
import pandas as pd
# Summary of performance metrics
performance_metrics = {
    'Model': ['BM25', 'VSM (BoW)', 'VSM (TF-IDF)'],
    'Precision': [bm25_precision, vsm_bow_precision, vsm_tfidf_precision],
    'Recall': [bm25_recall, vsm_bow_recall, vsm_tfidf_recall],
    'MAP': [mean_average_precision, mean_average_precision, mean_average_precision]  # Assuming MAP is the same for all models
}

# Convert the performance metrics to a DataFrame for better visualization
performance_df = pd.DataFrame(performance_metrics)

# Display the performance metrics
print(performance_df)

# Conclusion 


In [60]:
# Summary of key findings, performance comparison, and insights

# Key Findings
key_findings = """
Key Findings:
1. The BM25 model, VSM with BoW, and VSM with TF-IDF all achieved the same precision and recall values.
2. The Mean Average Precision (MAP) for all models was also the same.
3. The models were able to retrieve relevant documents effectively, with BM25 and VSM models showing similar performance.
"""

# Performance Comparison
performance_comparison = performance_df.to_string(index=False)

# Insights
insights = """
Insights:
1. The precision and recall values indicate that the models are effective in retrieving relevant documents.
2. The similarity in performance metrics suggests that all three models are equally capable of handling the given dataset.
3. The choice of model may depend on other factors such as computational efficiency and ease of implementation.
4. Visualization tools such as word clouds and frequency distributions provide valuable insights into the text data.
5. LDA topic modeling can help in understanding the underlying themes in the documents.
"""

# Print the summary
print(key_findings)
print("\nPerformance Comparison:\n")
print(performance_comparison)
print("\n" + insights)


Key Findings:
1. The BM25 model, VSM with BoW, and VSM with TF-IDF all achieved the same precision and recall values.
2. The Mean Average Precision (MAP) for all models was also the same.
3. The models were able to retrieve relevant documents effectively, with BM25 and VSM models showing similar performance.


Performance Comparison:

       Model  Precision  Recall      MAP
        BM25        0.6    0.75 0.358333
   VSM (BoW)        0.6    0.75 0.358333
VSM (TF-IDF)        0.6    0.75 0.358333


Insights:
1. The precision and recall values indicate that the models are effective in retrieving relevant documents.
2. The similarity in performance metrics suggests that all three models are equally capable of handling the given dataset.
3. The choice of model may depend on other factors such as computational efficiency and ease of implementation.
4. Visualization tools such as word clouds and frequency distributions provide valuable insights into the text data.
5. LDA topic modeling can 

# Future Work and Areas for Improvement

1. **Dataset Expansion**: Incorporate a larger and more diverse dataset to evaluate the models' performance on a wider range of topics and document types.

2. **Advanced Preprocessing Techniques**: Explore additional preprocessing techniques such as named entity recognition (NER), part-of-speech (POS) tagging, and dependency parsing to enhance text representation.

3. **Hyperparameter Tuning**: Perform hyperparameter tuning for the retrieval models (e.g., BM25 parameters, vectorizer settings) to optimize their performance.

4. **Incorporate Deep Learning Models**: Investigate the use of deep learning-based retrieval models such as BERT, GPT, or other transformer-based models to improve retrieval accuracy.

5. **User Feedback Integration**: Develop a mechanism to incorporate user feedback into the retrieval system to continuously improve its relevance and accuracy.

6. **Multilingual Support**: Extend the system to support multiple languages, enabling retrieval from multilingual datasets.

7. **Real-time Retrieval**: Implement real-time retrieval capabilities to handle dynamic and continuously updating datasets.

8. **Scalability and Performance**: Optimize the system for scalability and performance to handle large-scale datasets and high query volumes efficiently.

9. **Evaluation Metrics**: Explore additional evaluation metrics such as F1-score, NDCG (Normalized Discounted Cumulative Gain), and user satisfaction to provide a more comprehensive assessment of model performance.

10. **Visualization Enhancements**: Enhance visualization tools to provide more interactive and insightful visualizations, such as interactive word clouds, topic modeling visualizations, and similarity heatmaps.

11. **Integration with External Systems**: Integrate the retrieval system with external systems or APIs to enable seamless access to external data sources and services.

12. **User Interface Improvements**: Develop a user-friendly interface to allow users to interact with the retrieval system more effectively, including advanced search options and result filtering.

By addressing these areas, the retrieval system can be further improved to provide more accurate, efficient, and user-friendly information retrieval capabilities.

# References

- **Datasets:**
    - 20 Newsgroups Dataset: [Kaggle - 20 Newsgroups](https://www.kaggle.com/crawford/20-newsgroups)

- **Tools and Libraries:**
    - **Pandas:** Data manipulation and analysis library. [Pandas Documentation](https://pandas.pydata.org/pandas-docs/stable/)
    - **NLTK:** Natural Language Toolkit for text processing. [NLTK Documentation](https://www.nltk.org/)
    - **Scikit-learn:** Machine learning library for Python. [Scikit-learn Documentation](https://scikit-learn.org/stable/)
    - **Rank-BM25:** Implementation of the BM25 ranking function. [Rank-BM25 GitHub](https://github.com/dorianbrown/rank_bm25)
    - **WordCloud:** A tool for generating word clouds. [WordCloud GitHub](https://github.com/amueller/word_cloud)
    - **PyLDAvis:** Interactive topic model visualization. [PyLDAvis GitHub](https://github.com/bmabey/pyLDAvis)
    - **Matplotlib:** Plotting library for Python. [Matplotlib Documentation](https://matplotlib.org/stable/contents.html)